## Byte Pair Encoding (BPE)

Subword Tokenizer란 텍스트 데이터를 하위 단어(subword)로 분리하여 처리하는 tokenization 기법

BPE는 자주 등장하는 문자 쌍을 반복적으로 병합하며, 점점 더 큰 단위(sub word)를 만들어냄 <br>
병합이 종료되면 고정된 크기의 subword 사전이 생성되고, 이 사전을 통해 텍스트를 토큰으로 나눌 수 있음

### Training Tokenizer

1. training corpus에 존재하는 모든 문자를 토큰으로 변환하여 시작
2. 가장 자주 등장하는 토큰 쌍을 찾아 하나의 토큰으로 병합
3. 원하는 크기의 (fixed)vocab_size에 도달할 때까지 반복

### Training BPE in Huggingface 
1. Initialization

먼저 BPE 모델을 사용하여 Tokenizer 객체를 생성함<br>
pre-tokenizer를 whitespace로 설정<br>
: 토크나이저가 BPE를 적용하기 전에 처음에는 공백을 기준으로 텍스트를 분할함 <br>

2. Training

vocab_size, min_frequency(어휘에 포함되기 위해 토큰이 가져야 할 최소 빈도)와 같은 특정 매개변수로 BpeTrainer를 정의<br>
Trainer에 데이터셋을 제공하여 학습 시킴


3. Tokenization

훈련 후, `encode`를 사용하여 텍스트 토크나이즈를 수행함<br>
출력 : 토크나이즈된 하위 단어, 해당 ID <br>
`decode`도 가능하지만 완벽한 재구성이 아닐 수 있음


In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [11]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(BPE())
# pre-tokenzier는 Whitespace : 공백을 기준으로 초기 설정
tokenizer.pre_tokenizer = Whitespace()

# BpeTrainer 매개 변수 설정
trainer = BpeTrainer(vocab_size=1000, min_frequency=2)

In [12]:
# Training data
data = ["Hello, how are you?", "I am fine, thank you.", "How about you?"]

# Train the tokenizer
tokenizer.train_from_iterator(data, trainer)

학습에 사용한 데이터 중 빈번하게 발생하는 하위 단어 'ow', 'you'를 확인할 수 있음

In [13]:
# Tokenize a new sentence
output = tokenizer.encode("Hello, how are you?")
print("Tokens:", output.tokens)

# Convert tokens to IDs
print("Input IDs:", output.ids)

# Decode IDs back to text
print("Decoded text:", tokenizer.decode(output.ids))

Tokens: ['H', 'e', 'l', 'l', 'o', ',', 'h', 'ow', 'a', 'r', 'e', 'you', '?']
Input IDs: [3, 7, 12, 12, 15, 0, 9, 23, 5, 16, 7, 22, 2]
Decoded text: H e l l o , h ow a r e you ?


In [14]:
output

Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

학습에 사용하지 않은 텍스트의 경우 토크나이징이 이뤄지지 않음

In [15]:
output = tokenizer.encode("안녕하세요")
print("Tokens:", output.tokens)
output

Tokens: []


Encoding(num_tokens=0, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

학습용 데이터를 불러와 토크나이저를 학습할 수 있음

또한 학습된 토크나이저는 `tokenzier.save`를 통해 json 파일로 저장하여 재사용 가능

In [16]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

# Initialize a BPE tokenizer
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()

# Normalize text (optional but recommended)
tokenizer.normalizer = normalizers.Sequence([NFD(), StripAccents()])

# Define a trainer for the tokenizer
trainer = BpeTrainer(vocab_size=10000, min_frequency=2, special_tokens=["<pad>", "<s>", "</s>", "<unk>"])

# Train the tokenizer using an input text file
files = ["test.txt"]
tokenizer.train(files, trainer)

# Save the tokenizer model for future use
tokenizer.save("bpe_tokenizer.json")

# Test the tokenizer on new text
encoded = tokenizer.encode("Hello, how are you?")
print("Tokens:", encoded.tokens)
print("IDs:", encoded.ids)

tokenizer.encode("안녕하세요").tokens




Tokens: ['H', 'e', 'l', 'l', 'o', 'o', 'w', 'a', 'r', 'e', 'y', 'o', 'u', '?']
IDs: [5, 7, 8, 8, 9, 9, 12, 6, 10, 7, 13, 9, 11, 4]


['ᄋ', 'ᅡ', 'ᆫ', 'ᄂ', 'ᅧ', 'ᆼ', 'ᄒ', 'ᅡ', 'ᄉ', 'ᅦ', '요']